<h2>Desarrollo del Desafio 2 - Machine Learning


<h3>Grupo 7

<li>Laura Jazmín Chao
<li>Juan Nicolás Capistrano
<li>Betiana Srur
<li>Ignacio Mendieta
<li>Marecelo Carrizo

**Desafio 2** : Aplicar las tecnicas de Machine Learning para calcular el precio por metro cuadrado de las propiedades.

In [100]:
import numpy as np
import pandas as pd

# Libreria Grafica.
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

# Libreria para analisis ML
from sklearn import datasets, linear_model

In [101]:
# Definimos la ruta de la información.
data_propiedades = "Data/properati.csv"

# Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")

# Chequeamos que los datos se hayan importado correctamente
data.sample(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
54381,54381,sell,house,Acacias Blancas,|Argentina|Bs.As. G.B.A. Zona Norte|Escobar|Ac...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,...,340.0,1323.529412,1323.529412,NaN,NaN,NaN,http://www.properati.com.ar/19mwx_venta_casa_a...,Amplia casa en doble lotes internos - Escobar ...,Bustamante Propiedades | Acacias Blancas en Venta,https://thumbs4.properati.com/1/13Iqn9orBO8eDm...
45979,45979,sell,apartment,Boedo,|Argentina|Capital Federal|Boedo|,Argentina,Capital Federal,3436003.0,"-34.6223745,-58.4091176",-34.622374,...,40.0,2395.348837,2575.000000,NaN,2.0,NaN,http://www.properati.com.ar/1959k_venta_depart...,Departamento 2 ambientes seminuevo al contrafr...,2amb Ctfte 43mts 7aos Categ San Juan y Urquiza,https://thumbs4.properati.com/5/0XlEnYNO6DqrQO...
113214,113214,sell,house,Banfield,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,3436152.0,"-34.7438273,-58.406684",-34.743827,...,272.0,1158.940397,1286.764706,NaN,7.0,NaN,http://www.properati.com.ar/1c81r_venta_casa_b...,"Venta de Casa 7 AMBIENTES O MAS en Banfield, L...",CASA EN VENTA,https://thumbs4.properati.com/8/yz7cpDo_97wis7...
37995,37995,sell,house,Castelar,|Argentina|Bs.As. G.B.A. Zona Oeste|Morón|Cast...,Argentina,Bs.As. G.B.A. Zona Oeste,3435607.0,"-34.7030503,-58.5887569",-34.703050,...,260.0,1166.666667,1346.153846,NaN,NaN,NaN,http://www.properati.com.ar/18kae_venta_casa_c...,CODIGO: ubicado en: Islas Malvinas 3100 - Pu...,"Chalet de categoria, Castelar Norte",https://thumbs4.properati.com/8/tZOaMwBEr9D74_...
85147,85147,sell,apartment,Córdoba,|Argentina|Córdoba|,Argentina,Córdoba,3860255.0,NaN,NaN,...,73.0,NaN,34356.164384,NaN,NaN,NaN,http://www.properati.com.ar/1b2nj_venta_depart...,"Departamento con Cloaca, Argentina, Córdoba, p...",Departamento - Nueva Cordoba,https://thumbs4.properati.com/5/46by_tDFqA4Ym5...


In [102]:
# Chequeamos cantidad de registros y cantidad de columnas
data.shape

(121220, 26)

<h2>Limpieza de Datos.

In [103]:
# Creamos una mask para traer los valores en monedas que no son USD ni ARS.
currency_OTHER_CURRENCY_mask = (data.currency == 'PEN') | (data.currency == 'UYU')
# Observamos esos registros
data.loc[currency_OTHER_CURRENCY_mask, :]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
50387,50387,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e8z_venta_depart...,Ref.: 55:PREVENTA ::: OPORTUNIDAD ::: Dúplex a...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/9/N_Qr_5X_vOq--b...
50388,50388,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e90_venta_depart...,Ref.: 56Duplex en venta de 1 dormitorio con ap...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/8/_cBXAyr_9pDySg...
107390,107390,sell,house,San Roque,|Argentina|Mendoza|San Roque|,Argentina,Mendoza,3836586.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1by0s_venta_casa_s...,"Casa con Calefacción central, Mendoza, San Roq...",Casa - San Vicente,https://thumbs4.properati.com/2/pA0RbuDKRK6GgV...


In [104]:
# Hacemos drop de esos registros y chequeamos la diferencia entre cantidad original y actual
cant_registros = data.shape[0]
data.drop(data.loc[currency_OTHER_CURRENCY_mask, :].index, inplace = True)
print(f'Cantidad de registros original - registro actuales: {cant_registros - data.shape[0]}')

Cantidad de registros original - registro actuales: 3


In [105]:
# Creamos la lista de columnas a filtrar
drop_columns = ['properati_url', 'Unnamed: 0', 'operation','image_thumbnail','expenses','lat-lon','geonames_id','lat', 'lon']

# Creamos una variable nueva para guardar los datos (el cambio se realiza inplace=False)
data_filtered = data.drop(drop_columns, axis=1)
data_filtered.head(5)

,property_type,place_name,place_with_parent_names,country_name,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description,title
0,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
4,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,1828.571429,NaN,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


**Esto no es necesario ahora ¿?**

In [106]:
# Creamos un diccionario con la nueva agrupación
provincia={'Capital Federal':'CABA','Bs.As. G.B.A. Zona Norte': 'Bs. As. G.B.A.', 'Bs.As. G.B.A. Zona Sur':'Bs. As. G.B.A.', 'Córdoba':'Córdoba','Santa Fe':'Santa Fe','Buenos Aires Costa Atlántica':'Buenos Aires', 'Bs.As. G.B.A. Zona Oeste':'Bs. As. G.B.A.', 'Buenos Aires Interior':'Buenos Aires','Río Negro':'Resto del país','Neuquén':'Resto del país', 'Mendoza':'Resto del país', 'Tucumán':'Resto del país', 'Corrientes':'Resto del país', 'Misiones':'Resto del país', 'Entre Ríos':'Resto del país', 'Salta':'Resto del país', 'Chubut':'Resto del país', 'San Luis':'Resto del país', 'La Pampa':'Resto del país','Formosa':'Resto del país','Chaco': 'Resto del país', 'San Juan':'Resto del país','Tierra Del Fuego':'Resto del país', 'Catamarca':'Resto del país', 'Jujuy': 'Resto del país', 'Santa Cruz':'Resto del país', 'La Rioja':'Resto del país', 'Santiago Del Estero':'Resto del país'}

In [107]:
# Mapeamos las provincias con el diccionario y ponermos el grupo en una columna nueva
data_filtered['province_group'] = data_filtered.state_name.map(provincia)
display(data_filtered.sample(5))


,property_type,place_name,place_with_parent_names,country_name,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description,title,province_group
29508,apartment,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,55000.0,USD,970447.50,55000.0,46.0,40.0,1195.652174,1375.000000,NaN,2.0,EXCELENTE OPORTUNIDAD.,OPORTUNIDAD 2 AMBIENTES ZONA GUEMES MAR DEL PLATA,Buenos Aires
81788,apartment,Garín,|Argentina|Bs.As. G.B.A. Zona Norte|Escobar|Ga...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Departamento, Escobar, Garín, por U$S 1",Departamento - Garin,Bs. As. G.B.A.
18994,apartment,Caballito,|Argentina|Capital Federal|Caballito|,Argentina,Capital Federal,94350.0,USD,1664758.57,94350.0,37.0,34.0,2550.000000,2775.000000,NaN,1.0,NUEVO !!! EMPRENDIMIENTO A ESTRENAR EN CABALLI...,EXCELENTE EMPRENDIMIENTO MONOAMBIENTES APTO PR...,CABA
13273,house,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,55000.0,USD,970447.50,55000.0,NaN,140.0,NaN,392.857143,1.0,2.0,Casa en Venta de 1 dorm. en Cordoba,"Venta CASA, Zona AEOROPUERTO!!!",Córdoba
37540,apartment,San Telmo,|Argentina|Capital Federal|San Telmo|,Argentina,Capital Federal,88000.0,USD,1552716.00,88000.0,NaN,43.0,NaN,2046.511628,NaN,1.0,Corredor Responsable: Ariel Champanier - CUCIC...,Depto Monoambiente en pleno corazón de San Telmo,CABA


<h2>Tomamos solamente CABA para el analisis.

In [108]:
# Creamos una máscara y la aplicamos al dataframe anterior para traer los registros que necesitamos
data_caba_mask = data_filtered.state_name == 'Capital Federal' 
data_caba = data_filtered.loc[data_caba_mask, :]

# Dropeamos la columna creada anteriormente.
data_caba.drop(columns=['province_group', 'country_name', 'state_name', 'place_with_parent_names','description', 'title'], inplace=True)

# Chequeamos cómo quedaron los datos
data_caba.head(3)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
0,PH,Mataderos,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN
2,apartment,Mataderos,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN
3,PH,Liniers,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
# Vemos la cantidad de datos solamente de CABA.
data_caba.shape

(32314, 12)

In [110]:
# Revisamos si hay barrios de caba mal indentificados "Ejemplo: Capital Federal"
pd.set_option('display.max_rows', 1000) # Para mostrar todas las filas.


In [111]:
# Dropeamos los de Capital Federal
capi_mask = data_caba.place_name == 'Capital Federal'
data_caba.drop(data_caba.loc[capi_mask, :].index, inplace=True)

In [112]:
data_caba.shape

(31019, 12)

In [113]:
data_caba.head(20)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
0,PH,Mataderos,62000.0,USD,1093959.00,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN
2,apartment,Mataderos,72000.0,USD,1270404.00,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN
3,PH,Liniers,95000.0,USD,1676227.50,95000.0,NaN,NaN,NaN,NaN,NaN,NaN
7,apartment,Belgrano,138000.0,USD,2434941.00,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN
8,apartment,Belgrano,195000.0,USD,3440677.50,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN
13,apartment,Palermo Soho,111700.0,USD,1970890.65,111700.0,50.0,30.0,2234.000000,3723.333333,NaN,1.0
14,apartment,Palermo Soho,147900.0,USD,2609621.55,147900.0,42.0,31.0,3521.428571,4770.967742,NaN,1.0
16,PH,Mataderos,239000.0,USD,4217035.50,239000.0,140.0,98.0,1707.142857,2438.775510,NaN,4.0
19,apartment,Palermo,350000.0,USD,6175575.00,350000.0,104.0,96.0,3365.384615,3645.833333,NaN,3.0
21,apartment,Palermo,270500.0,USD,4772837.25,270500.0,118.0,73.0,2292.372881,3705.479452,NaN,4.0


## De los datos filtrados según Desafio 1 deberiamos:

1. Terminar de dropear variables incompletas que serian necesarias para poder entrenar un modelo. (ejemplo: si la propiedad no posee: price, price_aprox_local_currency,price_usd_per_m2, price_per_m2, etc...) la información de los barrios ya fue analizado y dropeado en el paso anterior.
2. Definir las variables CUANTITATIVAS, ejemplo precio, total de m2 de superficie.
3. Armar las variables CUALITATICAS, Dummy (ejemplo: place_name, property_type).
4. Armar una nueva variable Dummy que contemple los precios de propiedad segun su distribución por metro cuadrado. Utilizando Panda CUT. 

## Finalizado el analisis de datos, armamos diferentes modelos para predecir el precio de la propiedad.

1. Regresion Lineal Multiple
2. Entrenar los modelos con Lasso, RidgeCV y utilizando Cross Validation.



### Primero damos homogenidad a los datos

Vamos a realizar ciertas verificaciones de los datos.

In [114]:
# 1. Verificamos que el price y el price_aprox_usd sean los mismos, y luego si fuera así dropeamos el price_aprox_usd

mask_price_validation = data_caba.price != data_caba.price_aprox_usd

display(data_caba.loc[mask_price_validation, :].shape)

# Detectamos que hay 4138 propiedades donde los valores son diferentes? pero estan en Pesos o son NaN?

display(data_caba.loc[mask_price_validation, :].sample(20))

# Detectamos que tenemos los dos casos, donde los precios son NaN y donde la moneda está en ARS (lo cual no estaría mal)
# En el caso de ARS, vemos que el valor de price toma el de la moneda local ~ Por lo que podriamos llegar a eliminar price, currency y price_aprox_local_currency 

# Para los valores de NaN tendriamos que tomar la decisión si los dropeamos o tratamos de calcular un precio promedio en base a la cantidad de metros cuadrados. Podemos hacerlo en base al calculo que realizamos en el Desafio 1 ??

(4138, 12)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
73255,apartment,Nuñez,NaN,NaN,NaN,NaN,55.0,NaN,NaN,NaN,NaN,NaN
20825,apartment,Parque Chacabuco,3359279.0,ARS,3324236.32,188400.71,NaN,116.0,NaN,28959.301724,2.0,3.0
120782,apartment,Palermo,NaN,NaN,NaN,NaN,52.0,50.0,NaN,NaN,NaN,NaN
84710,apartment,Palermo,NaN,NaN,NaN,NaN,130.0,112.0,NaN,NaN,NaN,NaN
1888,apartment,Monte Castro,NaN,NaN,NaN,NaN,37.0,33.0,NaN,NaN,NaN,NaN
78330,apartment,Belgrano,5122951.0,ARS,5069510.46,287313.92,60.0,52.0,4788.565333,98518.288462,NaN,NaN
118036,apartment,Villa Crespo,NaN,NaN,NaN,NaN,83.0,64.0,NaN,NaN,NaN,NaN
1798,apartment,Almagro,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN
89936,apartment,Palermo Soho,NaN,NaN,NaN,NaN,75.0,67.0,NaN,NaN,NaN,NaN
88053,apartment,Villa Urquiza,NaN,NaN,NaN,NaN,40.0,31.0,NaN,NaN,NaN,NaN


In [115]:
# Elimino primero los registros donde no tengo nada de informacion. 

umbral = 10
data_caba.dropna(axis = 0, thresh=umbral, inplace=True)
# OJO que utilizando umbral de 10 se fueron casi 10000 datos incompletos.

# Vuelvo a ver como quedaron los datos
mask_price_validation = data_caba.price != data_caba.price_aprox_usd
display(data_caba.loc[mask_price_validation, :].shape)

display(data_caba.shape)
display(data_caba.loc[mask_price_validation, :].sample(20))


(1035, 12)

(23075, 12)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
19479,apartment,Flores,1700000.0,ARS,1682266.33,95342.25,48.0,45.0,1986.296875,37777.777778,1.0,2.0
92381,apartment,Abasto,1564000.0,ARS,1547685.02,87714.87,65.0,60.0,1349.459538,26066.666667,NaN,NaN
91665,apartment,Belgrano,7316549.0,ARS,7240225.77,410338.96,114.0,97.0,3599.464561,75428.340206,NaN,NaN
120404,apartment,San Telmo,1456400.0,ARS,1441207.34,81680.26,38.0,35.0,2149.480526,41611.428571,NaN,NaN
35639,apartment,San Cristobal,2872670.0,ARS,2842703.45,161109.89,86.0,86.0,1873.370814,33403.139535,NaN,NaN
42263,apartment,Almagro,1363000.0,ARS,1348781.75,76442.05,31.0,31.0,2465.872581,43967.741935,NaN,2.0
112333,apartment,San Telmo,1234400.0,ARS,1221523.26,69229.69,34.0,31.0,2036.167353,39819.354839,NaN,NaN
118761,apartment,Villa Urquiza,1821887.0,ARS,1802881.83,102178.12,35.0,32.0,2919.374857,56933.968750,NaN,NaN
120387,apartment,San Telmo,1275500.0,ARS,1262194.36,71534.72,34.0,31.0,2103.962353,41145.161290,NaN,NaN
120376,apartment,San Telmo,1419600.0,ARS,1404791.21,79616.38,34.0,31.0,2341.658235,45793.548387,NaN,NaN


In [116]:
# Para la superficie de metros cuadadros totales, podemos creer que en caso de ser Nan es porque es un departamente y la cantidad de m2 cubierta sería iguala la cantidad de m2 totales.data_caba


data_caba.surface_total_in_m2.fillna(data_caba.surface_covered_in_m2, inplace=True)

data_caba.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
117312,apartment,Coghlan,101100.0,USD,1783858.95,101100.00,38.0,33.0,2660.526316,3063.636364,NaN,NaN
114434,apartment,Paternal,95000.0,USD,1676227.50,95000.00,56.0,53.0,1696.428571,1792.452830,NaN,3.0
70634,apartment,Constitución,76400.0,USD,1348039.80,76400.00,30.0,26.0,2546.666667,2938.461538,NaN,NaN
50097,apartment,Nuñez,240000.0,USD,4234680.00,240000.00,55.0,50.0,4363.636364,4800.000000,NaN,2.0
12298,apartment,Villa Luro,300000.0,USD,5293350.00,300000.00,150.0,150.0,2000.000000,2000.000000,1.0,4.0
113435,apartment,Puerto Madero,550000.0,USD,9704475.00,550000.00,89.0,89.0,6179.775281,6179.775281,NaN,3.0
48848,apartment,San Telmo,38205.0,USD,674108.12,38205.00,38.0,32.0,1005.394737,1193.906250,NaN,1.0
88526,store,Villa del Parque,399000.0,ARS,394837.68,22377.38,246.0,246.0,90.964959,1621.951220,NaN,NaN
78880,apartment,Belgrano,115000.0,USD,2029117.50,115000.00,37.0,30.0,3108.108108,3833.333333,NaN,NaN
41484,apartment,Monte Castro,88000.0,USD,1552716.00,88000.00,40.0,NaN,2200.000000,NaN,7.0,1.0


In [117]:
# Idem al punto anterior pero ahora de surface_total a surface_covered

data_caba.surface_covered_in_m2.fillna(data_caba.surface_total_in_m2, inplace=True)

data_caba.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
62848,apartment,Centro / Microcentro,78141.0,USD,1378758.87,78141.0,32.0,27.0,2441.906250,2894.111111,NaN,1.0
52213,apartment,Nuñez,480000.0,USD,8469360.00,480000.0,282.0,282.0,NaN,1702.127660,7.0,5.0
87961,apartment,Parque Patricios,121482.0,USD,2143489.14,121482.0,60.0,41.0,2024.700000,2962.975610,NaN,NaN
83749,apartment,Puerto Madero,2950000.0,USD,52051275.00,2950000.0,460.0,354.0,6413.043478,8333.333333,NaN,NaN
82707,PH,Villa Pueyrredón,220000.0,USD,3881790.00,220000.0,120.0,100.0,1833.333333,2200.000000,NaN,NaN
60720,house,Villa Urquiza,360000.0,USD,6352020.00,360000.0,195.0,158.0,1846.153846,2278.481013,NaN,4.0
88387,PH,Flores,135000.0,USD,2382007.50,135000.0,70.0,65.0,1928.571429,2076.923077,NaN,NaN
52009,store,Parque Patricios,650000.0,USD,11468925.00,650000.0,141.0,141.0,4609.929078,4609.929078,NaN,NaN
80052,apartment,Almagro,120000.0,USD,2117340.00,120000.0,73.0,52.0,1643.835616,2307.692308,NaN,NaN
32973,store,Balvanera,1200000.0,USD,21173400.00,1200000.0,216.0,216.0,5555.555556,5555.555556,NaN,NaN


In [118]:
# Idem al punto anterior pero ahora de surface_total a surface_covered

data_caba.price_usd_per_m2.fillna(data_caba.price_aprox_usd / data_caba.surface_total_in_m2, inplace=True)

data_caba.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
80708,apartment,Flores,97200.0,USD,1715045.4,97200.0,46.0,43.0,2113.043478,2260.465116,NaN,NaN
26001,apartment,Boedo,37000.0,USD,652846.5,37000.0,30.0,30.0,1233.333333,1233.333333,NaN,1.0
22000,apartment,Caballito,330000.0,USD,5822685.0,330000.0,180.0,180.0,1833.333333,1833.333333,9.0,4.0
45931,apartment,Palermo,350000.0,USD,6175575.0,350000.0,130.0,90.0,2692.307692,3888.888889,NaN,NaN
93160,PH,Villa Urquiza,159000.0,USD,2805475.5,159000.0,84.0,84.0,1892.857143,1892.857143,NaN,NaN
80468,apartment,Saavedra,188000.0,USD,3317166.0,188000.0,85.0,59.0,2211.764706,3186.440678,NaN,NaN
113514,apartment,Palermo,700000.0,USD,12351150.0,700000.0,90.0,90.0,7777.777778,7777.777778,NaN,3.0
93926,apartment,Palermo,245000.0,USD,4322902.5,245000.0,60.0,60.0,4083.333333,4083.333333,NaN,NaN
55242,apartment,Coghlan,145000.0,USD,2558452.5,145000.0,71.0,71.0,2042.253521,2042.253521,NaN,4.0
73785,apartment,Caballito,140000.0,USD,2470230.0,140000.0,58.0,48.0,2413.793103,2916.666667,NaN,NaN


In [119]:
display(data_caba.floor.count())

# El piso del departamente es un dato muy incompleto por lo que se dropea

# Dropeamos la columna creada anteriormente.
data_caba.drop(columns=['floor'], inplace=True)

# En el caso de Rooms lo reemplazamos por 0
data_caba.rooms.fillna(0, inplace=True)

2816

In [120]:
display(data_caba.rooms.count())

# En el caso de rooms, podemos tomar que el dato para los NaN es 0?? modifica esto el analisis para predecir? 

23075

In [121]:
# Dropeamos los valores que sabemos que está repetidos o con otra moneda.data_caba

data_caba.drop(columns=['price', 'currency','price_per_m2'], inplace=True)

data_caba.sample(20)

,property_type,place_name,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,rooms
26962,house,Villa Luro,6352020.00,360000.00,300.0,200.0,1200.000000,5.0
79839,apartment,Retiro,9175140.00,520000.00,101.0,95.0,5148.514851,0.0
100832,house,Flores,4411125.00,250000.00,191.0,135.0,1308.900524,0.0
57993,apartment,Recoleta,1517427.00,86000.00,31.0,31.0,2774.193548,1.0
76740,apartment,Villa Luro,4852237.50,275000.00,195.0,72.0,1410.256410,0.0
60778,apartment,Floresta,935158.50,53000.00,22.0,19.0,2409.090909,1.0
111111,apartment,Belgrano,5434506.00,308000.00,87.0,77.0,3540.229885,3.0
7092,apartment,Recoleta,2099695.50,119000.00,40.0,40.0,2975.000000,2.0
62097,apartment,Almagro,1093959.00,62000.00,30.0,30.0,2066.666667,1.0
78454,apartment,Flores,1446849.00,82000.00,50.0,43.0,1640.000000,0.0


In [122]:
print('\nPorcentaje de valores incompletos por columna:')
print(f'{round(100 * data_caba.isnull().sum()/data_caba.shape[0], 2)}')

display(data_caba.shape)


Porcentaje de valores incompletos por columna:
property_type                 0.0
place_name                    0.0
price_aprox_local_currency    0.0
price_aprox_usd               0.0
surface_total_in_m2           0.0
surface_covered_in_m2         0.0
price_usd_per_m2              0.0
rooms                         0.0
dtype: float64


(23075, 8)

### Falta reindexar las filas.

Dummies de property_type y place_name, no se si conviene realizar el trabajo de la extracción de rooms, hay que evaluar el esfuerzo.

